In [0]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [0]:
#Go into main website, retrieve all sub websites
source_path = "https://www.givingbackassoc.org/english/charities-database/laos/"
page = requests.get(source_path)
page_content = page.content
soup = BeautifulSoup(page_content, 'html.parser')

In [0]:
# Specific Finders

def telephone_finder(data):
    key_phrases = ['TEL:', 'TEL :', 'TELEPHONE:', 'TELEPHONE :', 'TEL.', 'TEL .', 'PHONE:', 'PHONE :']

    for phrase in key_phrases:
        
        if phrase in data.upper():
            try:
                numbers_withstuff = data.upper().split(phrase)[1][:20]
                numbers_withstuff = re.sub("\D", "", numbers_withstuff)
                return numbers_withstuff
            except:
                pass

def fax_finder(data):
    key_phrases = ['FAX:', 'FAX :']

    for phrase in key_phrases:
        
        if phrase in data.upper():
            try:
                numbers_withstuff = data.upper().split(phrase)[1][:20]
                numbers_withstuff = re.sub("\D", "", numbers_withstuff)
                return numbers_withstuff
            except:
                pass
            
def email_finder(data):
    key_phrases = ['EMAIL:', 'EMAIL :']

    for phrase in key_phrases:
        
        if phrase in data.upper():
            try:
                numbers_withstuff = data.upper().split(phrase)[1][:45]
                numbers_withstuff = numbers_withstuff.lower()
                end = numbers_withstuff.find(".org")
                return numbers_withstuff.lower()[:end+4]
            except:
                pass

In [0]:
df = pd.DataFrame()
for i,s in enumerate(soup.select('div.j-module.n.j-hgrid')):
    df.set_value(i,'Name', s.select('div.j-module.n.j-text > p')[0].find('span').get_text())
    df.set_value(i,'Website', s.select('div.j-module.n.j-text span > a')[0].get('href'))
    temp = ""
    for j in range(3,len(s.select('div.j-module.n.j-text > p'))):
        temp = temp + s.select('div.j-module.n.j-text > p')[j].get_text(strip=True)
    df.set_value(i,'Description', temp) 

# Run Through Finders  
df['Telephone'] = df['Description'].apply(telephone_finder)
df['Fax'] = df['Description'].apply(fax_finder)
df['Email'] = df['Description'].apply(email_finder)


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


In [0]:
df

,Name,Website,Description,Telephone,Fax,Email
0,The Asia Foundation,http://asiafoundation.org/,The Asian Foundation opened its first office i...,4159824640,4153928863,None
1,Community Learning International,http://www.communitylearninginternational.org/,Community Learning Internationalis deeply comm...,02055510575,None,info@communitylearninginternational.org
2,GreenHeart Foundation,http://www.greenheartfoundation.org/,The GreenHeart Foundation is dedicated to Lao ...,None,None,None
3,Lao Rehabilitation Foundation (LRF),http://lao-foundation.org/,The main purpose of Lao Rehabilitation Foundat...,7072656555,7072656566,None
4,Laos,http://www.leot.org.uk/,Laos Educational Oportunities Trust (LEOT) is ...,None,None,None
5,Education for Development Foundation Japan,http://www.minsai.org/,MinsaiCenter is a Tokyo-based international no...,0364575782,None,None
6,SEDA-Laos,http://seda-laos.org/,The Social and Economic Developers Association...,85621314338,None,None
7,SOS Children's Villages,http://www.soschildrensvillages.org.uk/,SOS Children's Villages is a large U.K.based c...,856217105448,None,None
8,Sustainable Laos Education Initiatives Inc.,http://www.sustainablelaos.org/,Sustainable Laos Education Initiatives Inc. is...,None,None,None
9,Travel to Teach (T2T),http://www.travel-to-teach.org/,Travel to Teach founded by volunteers in 2002 ...,None,None,None


In [0]:
df.to_csv("givebackassoclaos.csv",header=True,index=False)